In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Prefer DUCKDB_PATH if set (same as consumer), else try common relative paths
env_path = os.getenv('DUCKDB_PATH')
candidates = [
    env_path,
    os.path.join('..', 'data', 'pipeline.duckdb'),
    os.path.join('data', 'pipeline.duckdb'),
]

resolved = None
for p in candidates:
    if p and Path(p).exists():
        resolved = p
        break

if not resolved:
    print("Could not find DuckDB file. Tried:")
    for p in candidates:
        if p:
            print(" -", os.path.abspath(p))
    print("Set DUCKDB_PATH or run the consumer with DuckDB sink:")
    print("  bash ../scripts/run_consumer.sh --window 60s --slide 10s --sink duckdb")
    raise SystemExit

print("Using DB:", os.path.abspath(resolved))
con = duckdb.connect(resolved, read_only=True)

exists = con.execute("select count(*) from duckdb_tables() where table_name = 'features'").fetchone()[0]
if exists == 0:
    print("Table 'features' not found. Available tables:")
    print(con.execute("select table_name from duckdb_tables() order by table_name").df())
    print("Run the consumer with DuckDB sink first, e.g.:")
    print("  bash ../scripts/run_consumer.sh --window 60s --slide 10s --sink duckdb")
    raise SystemExit

df = con.execute("""
select symbol, window_end as ts, log_return, volatility, num_ticks
from features
order by ts
""").df()

fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
for sym, g in df.groupby('symbol'):
    g.plot(x='ts', y='log_return', ax=ax[0], label=sym)
    g.plot(x='ts', y='volatility', ax=ax[1], label=sym)
ax[0].set_title('Windowed log returns')
ax[1].set_title('Windowed volatility')
plt.tight_layout()
plt.show()
